In [53]:
import time
import queue
import random
import numpy as np

In [67]:
class Board:

    def __init__(self, N=4):
        """ Initialize a square board, of size NxN, with a single empty spot.
        
        Parameters
        ----------
            N: Integer, size of the side of a square board.
        
        Edited by Miguel Agueda-Cabral
        """
        
        self.N = N  # Store size of board for later instantiations.
        self.board = np.arange(1, N**2 + 1)  # Use numpy to create a 1-D array with elements from 1->(N**2), inclusive. 
        self.board = self.board.reshape((N, N)).tolist()  # Reshape 1-D array into 2-D array of size NxN. 
        self.board[-1][-1] = '*'  # Insert blank spot at last row and last column intersection.
        
        self.goal = []
        for i in self.board:
            self.goal.append(tuple(i))
        self.goal = tuple(self.goal)  # Unaltered puzzle, already in Goal-state.
        self.empty = [N-1, N-1]

    def __repr__(self):
        """ Format puzzle board in neat and understandable format."""
        
        string = ''
        for row in self.board:
            for num in row:
                if len(str(num)) == 1:
                    string += '   ' + str(num)
                elif len(str(num)) > 1:
                    string += '  ' + str(num)
            string += '\n'
        return string

    def convert_to_tuple(self, ar):
        result = []
        for i in ar:
            result.append(tuple(i))
        return tuple(result)

    def convert_to_list(self, tup):
        result = []
        for i in tup:
            result.append(list(i))
        return result

    def match(self, copy):  # Locate empty spot in puzzle.
        a = Board(self.N)  # Create new Board object.
        a.board = copy  # Set board attribute to `copy`.
        for row in range(0, len(a.board)):
            for col in range(0, len(a.board[0])):
                if a.board[row][col] == '*':  # Search for asterisk.
                    a.empty = [row, col]  # When found, store the location of empty spot as attribute.
        result = []
        for i in a.board:
            result.append(list(i))
        a.board = result  # Convert copy of board to list format.
        return a

    def move_up(self): # move empty block up
        try:
            if self.empty[0] != 0:
                tmp = self.board[self.empty[0]-1][self.empty[1]]
                self.board[self.empty[0]-1][self.empty[1]] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0]-1, self.empty[1]]
        except IndexError:
            pass

    def move_down(self): # move empty block down
        try:
            tmp = self.board[self.empty[0]+1][self.empty[1]]
            self.board[self.empty[0]+1][self.empty[1]] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0]+1, self.empty[1]]
        except IndexError:
            pass

    def move_right(self): # move empty block right
        try:
            tmp = self.board[self.empty[0]][self.empty[1]+1]
            self.board[self.empty[0]][self.empty[1]+1] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0], self.empty[1]+1]
        except IndexError:
            pass

    def move_left(self): # move empty block left
        try:
            if self.empty[1] != 0:
                tmp = self.board[self.empty[0]][self.empty[1]-1]
                self.board[self.empty[0]][self.empty[1]-1] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0], self.empty[1]-1]
        except IndexError:
            pass

    def shuffle(self, steps):
        for i in range(0, steps):
            direction = random.randrange(1, 5)
            if direction == 1:
                self.move_up()
            elif direction == 2:
                self.move_right()
            elif direction == 3:
                self.move_left()
            elif direction == 4:
                self.move_down()

    def solve_bfs(self):
        start = self.convert_to_tuple(self.board)
        pred = {}
        visited = []
        frontier = queue.Queue()
        frontier.put(start)
        
        while frontier.qsize() > 0:
            # print(F"Size of Frontier: {frontier.qsize()}")
            tmp = frontier.get()
            
            if tmp == self.goal:  # Check if the puzzle is solved.
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:  # Otherwise, check if the puzzle is in a novel state.
                # If puzzle is in novel state, expand all possible movements as new puzzles.
                visited.append(tmp)  # Store a copy of tmp in visited list.
                tmpboard = self.match(tmp)  # Generate temporary board from tmp.
                tmpboard.move_up()  # Move empty spot up,
                if self.convert_to_tuple(tmpboard.board) != tmp:  # Check if it is unchanged.
                    frontier.put(self.convert_to_tuple(tmpboard.board))  # If it is changed, push to frontier.
                    if self.convert_to_tuple(tmpboard.board) not in pred:  # Check if board already has a predicted movement,
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']  # If it does not, add it to predictions.

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')

    def solve_dfs(self):
        """ Solve puzzle board using Depth First Search.
        
        Parameters
        ----------
            None
        
        Returns
        -------
            path: List of steps to take to get from Start state to Goal state.
            
        Forked function from solve_bfs, edited by Miguel Agueda-Cabral.
        """
        
        start = self.convert_to_tuple(self.board)
        pred = {}
        visited = []
        frontier = queue.LifoQueue()  # The only thing needed to convert from BFS to DFS was to use LIFO. 
        frontier.put(start)
        
        while frontier.qsize() > 0:
            # print(F"Size of Frontier: {frontier.qsize()}")
            tmp = frontier.get()
            
            if tmp == self.goal:  # Check if the puzzle is solved.
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:  # Otherwise, check if the puzzle is in a novel state.
                # If puzzle is in novel state, expand all possible movements as new puzzles.
                visited.append(tmp)  # Store a copy of tmp in visited list.
                tmpboard = self.match(tmp)  # Generate temporary board from tmp.
                tmpboard.move_up()  # Move empty spot up,
                if self.convert_to_tuple(tmpboard.board) != tmp:  # Check if it is unchanged.
                    frontier.put(self.convert_to_tuple(tmpboard.board))  # If it is changed, push to frontier.
                    if self.convert_to_tuple(tmpboard.board) not in pred:  # Check if board already has a predicted movement,
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']  # If it does not, add it to predictions.

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')

In [65]:
def time_it(f):
    """ Measure total execution time required by a function, f.
    
    Parameters
    ----------
        f: Function call to execute while measuring time elapsed.
    
    Returns
    -------
        time_elapsed: Total time required to execute function, f, in units of Second.
        res: Output of the function, f. 
        
    Authored by Miguel Agueda-Cabral.
    """
    
    start_t = time.time()
    res = f()
    time_elapsed = time.time() - start_t
    return time_elapsed, res

In [76]:
""" Edited by Miguel Agueda-Cabral"""
N = int(input("How many pieces per side: "))  # Set number of puzzle pieces per side. 
board = Board(N)  # Initialize board of size NxN. 
print("Unaltered Board\n", board)
board.shuffle(15)
print ("Shuffled Board\n", board)

bfs_time, path = time_it(board.solve_bfs)
print(F"BFS Path: {path}\nTook {bfs_time} seconds to compute.")
dfs_time, path = time_it(board.solve_dfs)
print (F"DFS Path: {path}\nTook {dfs_time} seconds to compute.")

for dir in path:
    if dir == 'right':
        board.move_right()
    if dir == 'left': 
        board.move_left()
    if dir == 'up':
        board.move_up()
    if dir == 'down':
        board.move_down()

print ("Solved Board\n", board)

How many pieces per side: 4
Unaltered Board
    1   2   3   4
   5   6   7   8
   9  10  11  12
  13  14  15   *

Shuffled Board
    1   2   3   4
   5   6   *   7
   9  10  11   8
  13  14  15  12

BFS Path: ['right', 'down', 'down']
Took 0.001999378204345703 seconds to compute.
DFS Path: ['left', 'left', 'down', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'down', 'left', 'left', 'up', 'left', 'down', 'right', 'right', 'right', 'up', 'left', 'left', 'left', 'down', 'right', 'right', 'right', 'up', 'left', 'left', 'left', 'down', 'right', 'right', 'right', 'up', 'left', 'left', 'left', 'down', 'right', 'up', 'right', 'r

Took 0.2279956340789795 seconds to compute.
Solved Board
    1   2   3   4
   5   6   7   8
   9  10  11  12
  13  14  15   *

